In [21]:
import cv2
import os
import csv
import numpy as np

def video_to_frames(video_path, output_folder, video_id, skip_seconds=20, interval_seconds=5):
    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    frames_to_skip = int(skip_seconds * fps)
    frames_per_interval = int(interval_seconds * fps)

    vidcap.set(cv2.CAP_PROP_POS_FRAMES, frames_to_skip)

    frames = []
    frame_number = frames_to_skip

    while True:
        success, image = vidcap.read()
        if not success:
            break

        if (frame_number - frames_to_skip) % frames_per_interval == 0:
            frame_filename = f"video_{video_id}_frame_{frame_number}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_path, image)

            frames.append({
                'frame_number': frame_number,
                'filename': frame_filename
            })

        frame_number += 1

    vidcap.release()
    return frames

def create_labels(num_frames):
    return np.linspace(0, 1, num_frames)

def process_videos(video_folder, output_folder):
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4') or f.endswith('.avi')]
    video_files.sort()  # Ensure consistent ordering
    
    labels = []
    
    for i, video_file in enumerate(video_files):
        video_path = os.path.join(video_folder, video_file)
        frames = video_to_frames(video_path, output_folder, i + 1)
        video_labels = create_labels(len(frames))
        
        for j, (frame, label) in enumerate(zip(frames, video_labels)):
            labels.append({
                'video_id': i + 1,
                'frame_number': frame['frame_number'],
                'filename': frame['filename'],
                'percentage': label
            })
    
    return labels

def save_labels_to_csv(labels, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['video_id', 'frame_number', 'filename', 'percentage']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for label in labels:
            writer.writerow(label)

# Main execution
video_folder = r'D:\Coding\Files\Project\Construction 2.0\Ml AI'
output_folder = r'D:\Coding\Files\Project\Construction 2.0\data processing (image)'
output_file = os.path.join(output_folder, 'labels.csv')

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

labels = process_videos(video_folder, output_folder)
save_labels_to_csv(labels, output_file)

print(f"Frames have been saved to {output_folder}")
print(f"Labels have been saved to {output_file}")

Frames have been saved to D:\Coding\Files\Project\Construction 2.0\data processing (image)
Labels have been saved to D:\Coding\Files\Project\Construction 2.0\data processing (image)\labels.csv


In [35]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Define constants
IMAGE_SIZE = (224, 224)  # MobileNetV2 default input size
BATCH_SIZE = 32
EPOCHS = 70  # Increased epochs, we'll use early stopping

def load_dataset(csv_file, image_folder):
    df = pd.read_csv(csv_file)
    X = []
    y = []
    
    for _, row in df.iterrows():
        img_path = os.path.join(image_folder, row['filename'])
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)  # Preprocess for MobileNetV2
        X.append(img_array)
        y.append(row['percentage'])
    
    return np.array(X), np.array(y)

def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    base_model.trainable = False  # Freeze the base model
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1)  # Output layer for regression
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',
                  metrics=['mae'])
    return model

def train_model(X_train, y_train, X_val, y_val):
    model = create_model()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
    
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        callbacks=[early_stopping, reduce_lr],
                        verbose=1)
    return model, history

def predict_completion(model, image_path):
    img = load_img(image_path, target_size=IMAGE_SIZE)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    return prediction

# Main execution
csv_file = r'D:\Coding\Files\Project\Construction 2.0\data processing (image)\labels.csv'
image_folder = r'D:\Coding\Files\Project\Construction 2.0\data processing (image)'

# Load and prepare the dataset
X, y = load_dataset(csv_file, image_folder)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the model
model, history = train_model(X_train, y_train, X_val, y_val)

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Mean Absolute Error: {test_mae:.4f}")

# Save the model
model.save('building_completion_model_enhanced.h5')

print("Enhanced model has been trained and saved as 'building_completion_model_enhanced.h5'")

Epoch 1/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 908ms/step - loss: 2.1338 - mae: 1.1481 - val_loss: 0.4721 - val_mae: 0.5691 - learning_rate: 0.0010
Epoch 2/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 715ms/step - loss: 1.6337 - mae: 1.0095 - val_loss: 0.2294 - val_mae: 0.3889 - learning_rate: 0.0010
Epoch 3/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 675ms/step - loss: 0.9951 - mae: 0.7767 - val_loss: 0.3610 - val_mae: 0.4977 - learning_rate: 0.0010
Epoch 4/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 690ms/step - loss: 0.9057 - mae: 0.7631 - val_loss: 0.3012 - val_mae: 0.4517 - learning_rate: 0.0010
Epoch 5/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 707ms/step - loss: 0.7448 - mae: 0.6914 - val_loss: 0.1797 - val_mae: 0.3523 - learning_rate: 0.0010
Epoch 6/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 694ms/step - loss: 0.6919 - mae: 0.6598 - val_loss: 0.1242 - val_mae: 0.2829 - learning_rate: 0.0010
Epoch 7/70
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 703ms/step - loss: 0.6243 - mae: 0.6333 - val_loss: 0.0944 - val_mae: 0.2424 - learning_rate: 0.0010

Test Mean Absolute Error: 0.0991
Enhanced model has been trained and saved as 'building_completion_model_enhanced.h5'


In [36]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

print("Training history plot has been saved as 'training_history.png'")

Training history plot has been saved as 'training_history.png'


In [38]:
# Example usage of the trained model
example_image_path = r'D:\Coding\Files\Project\Construction 2.0\WhatsApp Image 2024-09-18 at 03.23.29_2f759c41.jpg'  # Replace with an actual image path
completion_percentage = predict_completion(model, example_image_path)
print(f"Predicted completion percentage: {completion_percentage:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted completion percentage: 43.07%


In [1]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Define constants
IMAGE_SIZE = (224, 224)  # Must match the size used during training
MODEL_PATH = r'D:\Coding\Files\Project\Construction 2.0\building_completion_model_enhanced.h5'  # Path to your saved model

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=IMAGE_SIZE)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

def predict_completion(model, image_path):
    processed_image = load_and_preprocess_image(image_path)
    prediction = model.predict(processed_image)[0][0]
    return prediction

def main():
    # Load the saved model
    model = load_model(MODEL_PATH)
    print(f"Model loaded from {MODEL_PATH}")

    while True:
        # Get image path from user
        image_path = input("\nEnter the path to the building image (or 'q' to quit): ")
        
        if image_path.lower() == 'q':
            break

        if not os.path.exists(image_path):
            print("Error: File does not exist. Please enter a valid file path.")
            continue

        try:
            # Make prediction
            completion_percentage = predict_completion(model, image_path)
            print(f"Predicted completion percentage: {completion_percentage:.2%}")
        except Exception as e:
            print(f"An error occurred: {str(e)}")

    print("Thank you for using the Building Completion Predictor!")

if __name__ == "__main__":
    main()

Model loaded from D:\Coding\Files\Project\Construction 2.0\building_completion_model_enhanced.h5
Thank you for using the Building Completion Predictor!
